# Filter datasets to ensure they are inline with eachother

In [1]:
from pathlib import Path
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import pandas as pd

### Identify images without a label

In [2]:
# project_dir = "D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data"

# image_files =  [file[:-4] for file in os.listdir(f"{project_dir}/images")]
# label_YOLOv8 = [file[:-4] for file in os.listdir(f"{project_dir}/labels_YOLOv8")]
# label_image =  [file[:-4] for file in os.listdir(f"{project_dir}/labels_as_images")]


In [4]:
# print(image_files[0])
# print("Images      ", len(image_files))
# print("Label_YOLOv8", len(label_YOLOv8))
# print("Label_image ", len(label_image))

2020-02-05_14.18.49--NAs--T1354-GFP_Burst_Frame_0_Granule_10
Images       596833
Label_YOLOv8 596833
Label_image  595553


In [5]:
# count = 0
# for im in image_files: # For every image
#     if (im in label_YOLOv8) and (im in label_image): # If image has both yolo label and image label
#         # Put valid image and labels into new folders
#         shutil.copy(f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/images/{im}.png", f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/FINAL_DATASET/images/{im}.png")
#         shutil.copy(f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/labels_as_images/{im}.png", f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/FINAL_DATASET/labels_as_images/{im}.png")
#         shutil.copy(f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/labels_YOLOv8/{im}.txt", f"D:/Master/MasterProject/dataset_creation/datasets/cutout_with_padding/all_data/FINAL_DATASET/labels_YOLOv8/{im}.txt")
#         count += 1
# print(f"Final dataset size is:", count)

# Create train, val, test datasets of given size

In [4]:
compiled_datasets_name = "compiled_datasets_16bit_large"

project_dir_FINAL = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding"
project_output_dir_FINAL = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding/{compiled_datasets_name}"

# Create folder structure
Path(project_output_dir_FINAL).mkdir(parents=True, exist_ok=True)
for s in ['train', 'val', 'test']:
    Path(project_output_dir_FINAL+f"/{s}").mkdir(parents=True, exist_ok=True)
    for st in ['images', 'labels']:
        Path(project_output_dir_FINAL+f"/{s}/{st}").mkdir(parents=True, exist_ok=True)

# image_files_FINAL  = [file[:-4] for file in os.listdir(f"{project_dir_FINAL}/images")]
img_directory = "images_grayscale_16bit" # "images"
image_files_FINAL  = [file[:-4] for file in os.listdir(f"{project_dir_FINAL}/{img_directory}")]
# label_YOLOv8_FINAL = [file[:-4] for file in os.listdir(f"{project_dir_FINAL}/labels_YOLOv8")]
label_image_FINAL  = [file[:-4] for file in os.listdir(f"{project_dir_FINAL}/labels_as_images")]

TOTAL_SIZE = 10000
subset_dataset = []
for i in np.random.choice(image_files_FINAL, TOTAL_SIZE, replace=False):
    subset_dataset.append(i)

x_train, x_test, y_train, y_test = train_test_split(subset_dataset, subset_dataset, test_size=0.3, random_state=1)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=1) # 0.25 x 0.8 = 0.2
print("--- Dataset sizes ---")
print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))
print(len(x_test))
print(len(y_test))

for im in x_train:
    shutil.copy(f"{project_dir_FINAL}/{img_directory}/{im}.png",  f"{project_output_dir_FINAL}/train/images/{im}.png")
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{im}.png", f"{project_output_dir_FINAL}/train/labels/{im}.png")

for im in x_val: 
    shutil.copy(f"{project_dir_FINAL}/{img_directory}/{im}.png",  f"{project_output_dir_FINAL}/val/images/{im}.png")
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{im}.png", f"{project_output_dir_FINAL}/val/labels/{im}.png")

for im in x_test: 
    shutil.copy(f"{project_dir_FINAL}/{img_directory}/{im}.png",  f"{project_output_dir_FINAL}/test/images/{im}.png")
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{im}.png", f"{project_output_dir_FINAL}/test/labels/{im}.png")


--- Dataset sizes ---
3500
3500
750
750
750
750


# Dividing dataset based on size

In [19]:
project_dir = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding/DATASET_DIVIDED_ON_SIZE"
# project_dir_FINAL = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding"

# # Create folder structure
# for s in [10,20,30,40,50,60,70,80,90]:
#     Path(project_dir+f"/{s}").mkdir(parents=True, exist_ok=True)
    
label_size_dataframe = {
    'label_name': [],
    'label_size': [],
}
# Only need labels
label_image_FINAL  = [file[:-4] for file in os.listdir(f"{project_dir_FINAL}/labels_as_images")]
for i in range(len(label_image_FINAL)):
    if i % 1000 == 0:
        print("Status:", i)
    feature_image = cv2.imread(f"{project_dir_FINAL}/labels_as_images/{label_image_FINAL[i]}.png", cv2.IMREAD_GRAYSCALE)
    label_size_dataframe['label_name'].append(label_image_FINAL[i])
    label_size_dataframe['label_size'].append(feature_image[feature_image == 1].size / (1024*1024))

pd.DataFrame(label_size_dataframe).to_csv(project_dir + "/label_size_dataframe.csv")


Status: 0
Status: 1000
Status: 2000
Status: 3000
Status: 4000
Status: 5000
Status: 6000
Status: 7000
Status: 8000
Status: 9000
Status: 10000
Status: 11000
Status: 12000
Status: 13000
Status: 14000
Status: 15000
Status: 16000
Status: 17000
Status: 18000
Status: 19000
Status: 20000
Status: 21000
Status: 22000
Status: 23000
Status: 24000
Status: 25000
Status: 26000
Status: 27000
Status: 28000
Status: 29000
Status: 30000
Status: 31000
Status: 32000
Status: 33000
Status: 34000
Status: 35000
Status: 36000
Status: 37000
Status: 38000
Status: 39000
Status: 40000
Status: 41000
Status: 42000
Status: 43000
Status: 44000
Status: 45000
Status: 46000
Status: 47000
Status: 48000
Status: 49000
Status: 50000
Status: 51000
Status: 52000
Status: 53000
Status: 54000
Status: 55000
Status: 56000
Status: 57000
Status: 58000
Status: 59000
Status: 60000
Status: 61000
Status: 62000
Status: 63000
Status: 64000
Status: 65000
Status: 66000
Status: 67000
Status: 68000
Status: 69000
Status: 70000
Status: 71000
Statu

In [30]:
# label_size_df = pd.read_csv("D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding\DATASET_DIVIDED_ON_SIZE\label_size_dataframe.csv", index_col=0)
# label_size_df = label_size_df[(0.1 < label_size_df['label_size']) & (label_size_df['label_size'] < 0.48)]
# pd.DataFrame(label_size_df).to_csv(project_dir + "/label_size_dataframe_with_deletions.csv")

# Split data based on label size

In [2]:
compiled_datasets_name = "compiled_datasets_16bit_medium_equal_label_size"

project_dir_FINAL = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding"
project_output_dir_FINAL = f"D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding/{compiled_datasets_name}"

# Create folder structure
Path(project_output_dir_FINAL).mkdir(parents=True, exist_ok=True)
for title in ['normal', 'gradient']:
    Path(project_output_dir_FINAL+f"/{title}").mkdir(parents=True, exist_ok=True) # Dataset folder

    for s in ['train', 'val', 'test']:
        Path(project_output_dir_FINAL+f"/{title}/{s}").mkdir(parents=True, exist_ok=True)
        for st in ['images', 'labels']:
            Path(project_output_dir_FINAL+f"/{title}/{s}/{st}").mkdir(parents=True, exist_ok=True)

label_size_df = pd.read_csv("D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding/EVEN_DATASET.csv", index_col=0) 
# label_size_df = label_size_df['label_name']
print(label_size_df.shape)
label_size_df.rename(columns={'index':'old_index'}, inplace=True)
print(label_size_df)

# Validation and Test. Sample from rest of verified dataframe
label_size_ALL_DATA = pd.read_csv("D:/Master/MasterProject/dataset_creation/datasets/FINAL_DATASET_cutout_with_padding/label_size_dataframe_with_deletions.csv", index_col=0)
old_shape = label_size_ALL_DATA.shape[0]
label_size_ALL_DATA.drop(index=label_size_df['old_index'], inplace=True)
# assert (old_shape - label_size_ALL_DATA.shape[0]) == label_size_df.shape[0], f"{(old_shape - label_size_ALL_DATA.shape[0])} == {label_size_df.shape[0]}"

VAL_TEST_SIZE = 1500
sampled_data = label_size_ALL_DATA.sample(VAL_TEST_SIZE, replace=False)['label_name']
x_test, x_val, _, _ = train_test_split(sampled_data, sampled_data, test_size=0.5, random_state=1)

# -------  Normal ------- 
# Create even training datasets for both. Populates train/labels and train/images
title, img_location = ('normal', "images_grayscale_16bit")
for img_name in label_size_df['label_name']:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png", f"{project_output_dir_FINAL}/{title}/train/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}.png",   f"{project_output_dir_FINAL}/{title}/train/images/{img_name}.png")
for img_name in x_test:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png", f"{project_output_dir_FINAL}/{title}/test/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}.png",   f"{project_output_dir_FINAL}/{title}/test/images/{img_name}.png")
for img_name in x_val:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png", f"{project_output_dir_FINAL}/{title}/val/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}.png",   f"{project_output_dir_FINAL}/{title}/val/images/{img_name}.png")

# -------  Gradient ------- 
# Create even training datasets for both. Populates train/labels and train/images 
title, img_location = ('gradient', "images_grayscale_16bit_gradient")
for img_name in label_size_df['label_name']:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png", f"{project_output_dir_FINAL}/{title}/train/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}_gradient.png",   f"{project_output_dir_FINAL}/{title}/train/images/{img_name}.png")    
for img_name in x_test:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png",        f"{project_output_dir_FINAL}/{title}/test/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}_gradient.png", f"{project_output_dir_FINAL}/{title}/test/images/{img_name}.png")
for img_name in x_val:
    # Copy label
    shutil.copy(f"{project_dir_FINAL}/labels_as_images/{img_name}.png",        f"{project_output_dir_FINAL}/{title}/val/labels/{img_name}.png")
    shutil.copy(f"{project_dir_FINAL}/{img_location}/{img_name}_gradient.png", f"{project_output_dir_FINAL}/{title}/val/images/{img_name}.png")

(3360, 3)
      old_index                                         label_name  label_size
0         66362  2020-02-05_14.36.40--NAs--T1354-GFP_Burst_Fram...    0.178321
1         62902  2020-02-05_14.36.40--NAs--T1354-GFP_Burst_Fram...    0.185653
2        104739  2020-02-05_14.38.46--NAs--T1354-GFP_Burst_Fram...    0.188028
3         31524  2020-02-05_14.35.36--NAs--T1354-GFP_Burst_Fram...    0.184062
4         45306  2020-02-05_14.35.36--NAs--T1354-GFP_Burst_Fram...    0.176215
...         ...                                                ...         ...
3355      86454  2020-02-05_14.37.45--NAs--T1354-GFP_Burst_Fram...    0.442201
3356      70655  2020-02-05_14.37.45--NAs--T1354-GFP_Burst_Fram...    0.429742
3357      74065  2020-02-05_14.37.45--NAs--T1354-GFP_Burst_Fram...    0.432381
3358      89427  2020-02-05_14.37.45--NAs--T1354-GFP_Burst_Fram...    0.441193
3359      47955  2020-02-05_14.35.36--NAs--T1354-GFP_Burst_Fram...    0.430477

[3360 rows x 3 columns]
